In [1]:
from google.colab import drive
# drive.mount('/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data')
drive.mount('/content/drive')

%cd drive/MyDrive/IITP/sohyun/creditcard_prediction/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data


In [2]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: Currently logged in as: sohyun. Use `wandb login --relogin` to force relogin


True

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import time
import datetime
from torch.autograd import grad
from torch.autograd import Variable
# from model import *
import matplotlib.pyplot as plt
# from utils import *
# from data_loader import *
import IPython
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score
import random
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

class Solver(object):
  DEFAULTS = {}   
  def __init__(self, device, data_loader, data_loader_v, data_loader_test, config):
    # Data loader
    self.__dict__.update(Solver.DEFAULTS, **config)

    self.data_loader = data_loader
    self.data_loader_v = data_loader_v
    self.data_loader_test = data_loader_test
    self.device = device

    # Build tensorboard if use
    self.build_model()
    if self.use_tensorboard:
      self.build_tensorboard()

    # Start with trained model
    if self.pretrained_model:
      self.load_pretrained_model()

  def build_model(self):
    # Define model
    self.dagmm = DaGMM(self.device, self.gmm_k)

    # Optimizers
    if self.wd :
      self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr, weight_decay=self.wd) #weight_decay=0.1
    else :
      self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr) #weight_decay=0.1

    # Print networks
    self.print_network(self.dagmm, 'DaGMM')

    if torch.cuda.is_available():
      self.dagmm.cuda()

  def print_network(self, model, name):
    num_params = 0
    for p in model.parameters():
      num_params += p.numel()
    print(name)
    # print(model)
    print("The number of parameters: {}".format(num_params))

  def load_pretrained_model(self):
    self.dagmm.load_state_dict(torch.load(os.path.join(
      self.model_save_path, '{}_dagmm.pth'.format(self.pretrained_model))))

    print("phi", self.dagmm.phi,"mu",self.dagmm.mu, "cov", self.dagmm.cov)
    print('loaded trained models (step: {})..!'.format(self.pretrained_model))

  # def build_tensorboard(self):
  #     from logger import Logger
  #     self.logger = Logger(self.log_path)

  def reset_grad(self):
    self.dagmm.zero_grad()

  def to_var(self, x, volatile=False):
    if torch.cuda.is_available():
      x = x.cuda()
    return Variable(x, volatile=volatile)

  def train(self):
    iters_per_epoch = len(self.data_loader)

    # Start with trained model if exists
    if self.pretrained_model : start = int(self.pretrained_model.split('_')[0])
    else : start = 0

    # Start training
    iter_ctr = 0
    start_time = time.time()
    best_f_score = -1
    self.ap_global_train = np.array([0,0,0])
    for e in range(start, self.num_epochs):
      for i, (input_data, labels) in enumerate(tqdm(self.data_loader)):
        iter_ctr += 1
        start = time.time()

        input_data = self.to_var(input_data)

        total_loss,sample_energy, recon_error, cov_diag = self.dagmm_step(input_data)
        
        # Logging
        loss = {}
        loss['total_loss'] = total_loss.data.item()
        loss['sample_energy'] = sample_energy.item()
        loss['recon_error'] = recon_error.item()
        loss['cov_diag'] = cov_diag.item()

        # Print out log info
        if (i+1) % self.log_step == 0:
          elapsed = time.time() - start_time
          total_time = ((self.num_epochs*iters_per_epoch)-(e*iters_per_epoch+i)) * elapsed/(e*iters_per_epoch+i+1)
          epoch_time = (iters_per_epoch-i)* elapsed/(e*iters_per_epoch+i+1)
          
          epoch_time = str(datetime.timedelta(seconds=epoch_time))
          total_time = str(datetime.timedelta(seconds=total_time))
          elapsed = str(datetime.timedelta(seconds=elapsed))

          lr_tmp = []
          for param_group in self.optimizer.param_groups:
            lr_tmp.append(param_group['lr'])
          tmplr = np.squeeze(np.array(lr_tmp))

          log = "Elapsed {}/{} -- {} , Epoch [{}/{}], Iter [{}/{}], lr {}".format(
            elapsed,epoch_time,total_time, e+1, self.num_epochs, i+1, iters_per_epoch, tmplr)

          for tag, value in loss.items():
            log += ", {}: {:.4f}".format(tag, value)

          # Save model checkpoints
          # if (i+1) % self.model_save_step == 0:
          #   torch.save(self.dagmm.state_dict(),
          #     os.path.join(self.model_save_path, '{}_{}_dagmm.pth'.format(e+1, i+1)))
      
      # epoch 마다
      accuracy, precision, recall, f_score = self.validation()
      wandb.log({"accuracy":accuracy,
              "precision":precision,
              "recall":recall,
              "f_score":f_score,
              "sample_energy":sample_energy.item(),
              "recpm_error":recon_error.item(),
              "cov_diag":cov_diag.item(),
              "total_loss":total_loss.data.item()})
      if f_score > best_f_score : 
        best_f_score = f_score
        torch.save(self.dagmm.state_dict(), os.path.join(self.model_save_path, '{}_{}_dagmm.pth'.format(e+1, i+1))) # Save model checkpoints
                
  def dagmm_step(self, input_data):
    self.dagmm.train()
    enc, dec, z, gamma = self.dagmm(input_data)
    total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma, self.lambda_energy, self.lambda_cov_diag)

    self.reset_grad()
    total_loss.backward()

    torch.nn.utils.clip_grad_norm_(self.dagmm.parameters(), 5)
    self.optimizer.step()

    return total_loss,sample_energy, recon_error, cov_diag

  def validation(self):
    self.dagmm.eval()
    N = 0; mu_sum = 0; cov_sum = 0; gamma_sum = 0

    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)
      
      batch_gamma_sum = torch.sum(gamma, dim=0)
      
      gamma_sum += batch_gamma_sum
      mu_sum += mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
      cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
      
      N += input_data.size(0)
        
    train_phi = gamma_sum / N
    train_mu = mu_sum / gamma_sum.unsqueeze(-1)
    train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)

    train_energy = []; train_labels = []; train_z = []
    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
      
      train_energy.append(sample_energy.data.cpu().numpy())
      train_z.append(z.data.cpu().numpy())
      train_labels.append(labels.numpy())

    train_energy = np.concatenate(train_energy,axis=0)
    train_z = np.concatenate(train_z,axis=0)
    train_labels = np.concatenate(train_labels,axis=0)

    # use validation data
    valid_energy = []; valid_labels = []; valid_z = []
    for it, (input_data, labels) in enumerate(self.data_loader_v):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)

      sample_energy, cov_diag = self.dagmm.compute_energy(z, size_average=False)
      valid_energy.append(sample_energy.data.cpu().numpy())
      valid_z.append(z.data.cpu().numpy())
      valid_labels.append(labels.numpy())

    valid_energy = np.concatenate(valid_energy,axis=0)
    valid_z = np.concatenate(valid_z,axis=0)
    valid_labels = np.concatenate(valid_labels,axis=0)

    combined_energy = np.concatenate([train_energy, valid_energy], axis=0)
    combined_labels = np.concatenate([train_labels, valid_labels], axis=0)

    thresh = np.percentile(combined_energy, 100 - 0.3)
    # print("Threshold :", thresh)

    pred = (valid_energy > thresh).astype(int)
    gt = valid_labels.astype(int)

    accuracy = accuracy_score(gt,pred)
    precision, recall, f_score, support = prf(gt, pred, average='macro', labels=[0,1])
    print("Accuracy : {:0.4f}, Precision : {:0.4f}, Recall : {:0.4f}, F-score : {:0.4f}".format(accuracy, precision, recall, f_score))
    return accuracy, precision, recall, f_score

  def test(self):
    print("======================TEST MODE======================")
    self.dagmm.eval()
    ### use train data
    N = 0
    mu_sum = 0
    cov_sum = 0
    gamma_sum = 0

    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)
      
      batch_gamma_sum = torch.sum(gamma, dim=0)
      
      gamma_sum += batch_gamma_sum
      mu_sum += mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
      cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
      
      N += input_data.size(0)
        
    train_phi = gamma_sum / N
    train_mu = mu_sum / gamma_sum.unsqueeze(-1)
    train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)

    # print("N:",N)
    # print("phi :\n",train_phi)
    # print("mu :\n",train_mu)
    # print("cov :\n",train_cov)

    train_energy = []
    train_labels = []
    train_z = []
    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
      
      train_energy.append(sample_energy.data.cpu().numpy())
      train_z.append(z.data.cpu().numpy())
      # train_labels.append(labels.numpy())

    train_energy = np.concatenate(train_energy,axis=0)
    train_z = np.concatenate(train_z,axis=0)
    # train_labels = np.concatenate(train_labels,axis=0)

    ### use validation data
    valid_energy = []
    valid_labels = []
    valid_z = []
    for it, (input_data, labels) in enumerate(self.data_loader_v):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, size_average=False)
      valid_energy.append(sample_energy.data.cpu().numpy())
      valid_z.append(z.data.cpu().numpy())
      # valid_labels.append(labels.numpy())

    valid_energy = np.concatenate(valid_energy,axis=0)
    valid_z = np.concatenate(valid_z,axis=0)
    # valid_labels = np.concatenate(valid_labels,axis=0)

    ### use test data
    test_energy = []
    test_labels = []
    test_z = []
    for it, (input_data, labels) in enumerate(self.data_loader_test):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, size_average=False)
      test_energy.append(sample_energy.data.cpu().numpy())
      test_z.append(z.data.cpu().numpy())
      # test_labels.append(labels.numpy())

    test_energy = np.concatenate(test_energy,axis=0)
    test_z = np.concatenate(test_z,axis=0)
    # test_labels = np.concatenate(test_labels,axis=0)

    # combine
    combined_energy = np.concatenate([train_energy, valid_energy, test_energy], axis=0)
    # combined_labels = np.concatenate([train_labels, valid_labels, test_labels], axis=0)

    thresh = np.percentile(combined_energy, 100 - 0.3)
    # print("Threshold :", thresh)

    pred = (test_energy > thresh).astype(int)
    # gt = test_labels.astype(int)

    # accuracy = accuracy_score(gt,pred)
    # precision, recall, f_score, support = prf(gt, pred, average='binary')
    # print("Accuracy : {:0.4f}, Precision : {:0.4f}, Recall : {:0.4f}, F-score : {:0.4f}".format(accuracy, precision, recall, f_score))
    # return accuracy, precision, recall, f_score
    return pred


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torch.autograd import Variable
import itertools
# from utils import *

class Cholesky(torch.autograd.Function):
  def forward(ctx, a):
    # l = torch.potrf(a, False)
    l = torch.cholesky(a, False)
    ctx.save_for_backward(l)
    return l
  def backward(ctx, grad_output):
    # l, = ctx.saved_variables
    l, = ctx.saved_tensors
    linv = l.inverse()
    inner = torch.tril(torch.mm(l.t(), grad_output)) * torch.tril(
        1.0 - Variable(l.data.new(l.size(1)).fill_(0.5).diag()))
    s = torch.mm(linv.t(), torch.mm(inner, linv))
    return s
    
class DaGMM(nn.Module):
  """Residual Block."""
  def __init__(self, device, n_gmm = 2, latent_dim=3):
    super(DaGMM, self).__init__()
    self.device = device

    # layers = []
    # layers += [nn.Linear(30, 20)]
    # layers += [nn.BatchNorm1d(20)]
    # layers += [nn.LeakyReLU()]
    # layers += [nn.Linear(20, 1)]
    # layers += [nn.BatchNorm1d(1)]
    # layers += [nn.LeakyReLU()]
    # self.encoder = nn.Sequential(*layers)
  
    # layers = []
    # layers += [nn.Linear(1, 20)]
    # layers += [nn.BatchNorm1d(20)]
    # layers += [nn.LeakyReLU()]
    # layers += [nn.Linear(20,30)]
    # self.decoder = nn.Sequential(*layers)

    layers = []
    layers += [nn.Linear(30,60)]
    # layers += [nn.BatchNorm1d(60)]
    layers += [nn.Tanh()]        
    layers += [nn.Linear(60,30)]
    # layers += [nn.BatchNorm1d(30)]
    layers += [nn.Tanh()]        
    layers += [nn.Linear(30,10)]
    # layers += [nn.BatchNorm1d(10)]
    layers += [nn.Tanh()]        
    layers += [nn.Linear(10,1)]
    self.encoder = nn.Sequential(*layers)

    layers = []
    layers += [nn.Linear(1,10)]
    # layers += [nn.BatchNorm1d(10)]
    layers += [nn.Tanh()]        
    layers += [nn.Linear(10,30)]
    # layers += [nn.BatchNorm1d(30)]
    layers += [nn.Tanh()]        
    layers += [nn.Linear(30,60)]
    # layers += [nn.BatchNorm1d(60)]
    layers += [nn.Tanh()]        
    layers += [nn.Linear(60,30)]
    self.decoder = nn.Sequential(*layers)

    layers = []
    layers += [nn.Linear(latent_dim,10)]
    layers += [nn.Tanh()]        
    layers += [nn.Dropout(p=0.5)]        
    layers += [nn.Linear(10,n_gmm)]
    layers += [nn.Softmax(dim=1)]
    self.estimation = nn.Sequential(*layers)

    self.register_buffer("phi", torch.zeros(n_gmm))
    self.register_buffer("mu", torch.zeros(n_gmm,latent_dim))
    self.register_buffer("cov", torch.zeros(n_gmm,latent_dim,latent_dim))

  def relative_euclidean_distance(self, a, b):
    return (a-b).norm(2, dim=1) / a.norm(2, dim=1)

  def forward(self, x):
    enc = self.encoder(x)
    dec = self.decoder(enc)

    rec_cosine = F.cosine_similarity(x, dec, dim=1)
    rec_euclidean = self.relative_euclidean_distance(x, dec)
    # z = torch.cat([rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)
    z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)
    
    gamma = self.estimation(z)
    return enc, dec, z, gamma

  def compute_gmm_params(self, z, gamma):
    N = gamma.size(0)
    sum_gamma = torch.sum(gamma, dim=0) # K
    phi = (sum_gamma / N) # K
    self.phi = phi.data

    mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1) # K x D
    self.mu = mu.data
    # z = N x D
    # mu = K x D
    # gamma N x K

    z_mu = (z.unsqueeze(1)- mu.unsqueeze(0)) # z_mu = N x K x D
    z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2) # z_mu_outer = N x K x D x D   
    cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim = 0) / sum_gamma.unsqueeze(-1).unsqueeze(-1) # K x D x D
    self.cov = cov.data
    return phi, mu, cov
      
  def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
    if phi is None : phi = self.phi #self.to_var(self.phi)
    if mu is None : mu = self.mu #self.to_var(self.mu)
    if cov is None : cov = self.cov # self.to_var(self.cov)

    k, D, _ = cov.size()
    z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

    cov_inverse = []
    det_cov = []
    cov_diag = 0
    eps = 1e-12
    for i in range(k):
      cov_k = cov[i] + (torch.eye(D)*eps).to(device=self.device) # K x D x D
      cov_inverse.append(torch.inverse(cov_k).unsqueeze(0))
      
      det_cov.append((Cholesky.apply(cov_k.cpu() * (2*np.pi)).diag().prod()).unsqueeze(0))
      cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

    cov_inverse = torch.cat(cov_inverse, dim=0) # K x D x D
    det_cov = torch.cat(det_cov).cuda() # K
    exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1) * cov_inverse.unsqueeze(0), dim=-2) * z_mu, dim=-1) # N x 
    max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0] # for stability (logsumexp)
    exp_term = torch.exp(exp_term_tmp - max_val)

    sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt(det_cov)).unsqueeze(0), dim = 1) + eps)

    if size_average : sample_energy = torch.mean(sample_energy)
    return sample_energy, cov_diag

  def to_var(x, volatile=False):
    if torch.cuda.is_available() : x = x.cuda()
    # return torch.tensor(x)
    return Variable(x, volatile=volatile)

  def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):
    recon_error = torch.mean((x - x_hat) ** 2)
    phi, mu, cov = self.compute_gmm_params(z, gamma)
    sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)
    loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag
    return loss, sample_energy, recon_error, cov_diag


class MyDataset(Dataset):
  def __init__(self, df, eval_mode):
    self.df = df
    self.eval_mode = eval_mode
    if self.eval_mode:
      self.labels = self.df['Class'].values
      self.df = self.df.drop(columns=['Class']).values
    else:
      self.df = self.df.values
      
  def __getitem__(self, index):
    if self.eval_mode:
      self.x = self.df[index]
      self.y = self.labels[index]
      return torch.Tensor(self.x), self.y
    else:
      self.x = self.df[index]
      return torch.Tensor(self.x), 0
  
  def __len__(self):
    return len(self.df)

# train_dataset = MyDataset(df=train_df, eval_mode=False)
# train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)

# val_dataset = MyDataset(df = val_df, eval_mode=True)
# val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)

import pandas as pd
def get_loader(data_path, batch_size, mode='train'):
  """Build and return data loader."""
  train_df = pd.read_csv('./train.csv')
  train_df = train_df.drop(columns=['ID'])
  val_df = pd.read_csv('./val.csv')
  val_df = val_df.drop(columns=['ID'])
  test_df = pd.read_csv('./test.csv')
  test_df = test_df.drop(columns=['ID'])

  #-------------------#
  #---# Normalize #---#
  #-------------------#
  from sklearn.preprocessing import StandardScaler
  # case 1 - standardscaler
  scaler_n = StandardScaler()
  scaler_n.fit(train_df.values)
  
  val_x = val_df.drop(columns=['Class'])
  train_x_scaleN = pd.DataFrame(scaler_n.transform(train_df.values), columns = train_df.columns) # 확인 : train_x_scaleN.mean(), train_x_scaleN.var()
  val_x_scaleN = pd.DataFrame(scaler_n.transform(val_x.values), columns = val_x.columns)
  test_x_scaleN = pd.DataFrame(scaler_n.transform(test_df.values), columns = test_df.columns)

  train_df = train_x_scaleN
  val_df = pd.concat([val_x_scaleN, pd.DataFrame(val_df['Class'])], axis=1)
  test_df = test_x_scaleN
  
  # dataset = MyDataset(data_path, mode)
  train_dataset = MyDataset(df=train_df, eval_mode=False)
  valid_dataset = MyDataset(df=val_df, eval_mode=True)
  test_dataset = MyDataset(df=test_df, eval_mode=False)
  
  shuffle = False
  if mode == 'train': shuffle = True

  data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=shuffle)
  data_loader_v = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=shuffle)
  data_loader_test = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=shuffle)
  return data_loader, data_loader_v, data_loader_test

In [134]:
import os
import argparse
# from solver import Solver
# from data_loader import get_loader
from torch.backends import cudnn
# from utils import *
import torch
def str2bool(v):
  return v.lower() in ('true')

def main(config):
  # For fast training
  cudnn.benchmark = True
  wandb.init()
  data_loader, data_loader_v, data_loader_test = get_loader(config.data_path, batch_size=config.batch_size, mode=config.mode)
  USE_CUDA = torch.cuda.is_available()
  device = torch.device('cuda:0' if USE_CUDA else 'cpu')

  # Solver
  solver = Solver(device, data_loader, data_loader_v, data_loader_test, vars(config))

  if config.mode == 'train' : 
    solver.train()
    return solver, 0
  elif config.mode == 'test' : 
    pred = solver.test()
    return solver, pred

if __name__ == '__main__':
  import easydict
  args = easydict.EasyDict({
      "num_epochs" : 100,
      "batch_size" : 8192,
      "gmm_k" : 2,
      "lambda_energy" : 0.1,
      "lambda_cov_diag" : 0.005,
      # "pretrained_model" : '',
      "pretrained_model" : None,
      "mode" : 'train',
      # "mode" : "test",
      "data_path" : "./",   
      "use_tensorboard" : False,
      "log_path" : './logs',
      "model_save_path" : './models',
      "log_step" : 100,
      "sample_step" : 194,
      "model_save_step" : 194,
      "lr" : 1e-4,
      "wd" : None
  })
  config = args
  # args = vars(config)
  print('------------ Options -------------')
  for k, v in sorted(args.items()):
    print('%s: %s' % (str(k), str(v)))
  print('-------------- End ----------------')

  solver, pred = main(config)

------------ Options -------------
batch_size: 8192
data_path: ./
gmm_k: 2
lambda_cov_diag: 0.005
lambda_energy: 0.1
log_path: ./logs
log_step: 100
lr: 0.0001
mode: train
model_save_path: ./models
model_save_step: 194
num_epochs: 100
pretrained_model: None
sample_step: 194
use_tensorboard: False
wd: None
-------------- End ----------------


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 1040, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 186, in setup
    tel.feature.set_init_tags = True
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/lib/telemetry.py", line 41, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 572, in _telemetry_callback
    self._telemetry_flush()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 583, in _telemetry_flush
    self._backend.interface._publish_telemetry(self._telemetry_obj)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 74, in _publish_telemetry
    self._publish(rec)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_queue.py", line 49, in _publish
    raise Exception("The w

Exception: ignored

In [73]:
sum(pred)

839

In [28]:
pred
submit = pd.read_csv('./sample_submission.csv')
submit['Class'] = pred
submit.to_csv('./submit_DAGMM_0.5_20220727.csv', index=False)

In [29]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
sum(pred)

1086